In [31]:
import time
from functools import lru_cache

# Memoization

Memoization is a technique for speeding up computation by preventing a function from running on the exact same inputs twice. It does so by storing the result of function calls, and returning them if the function is called again with the same inputs.
In this recitation, we'll explore memoization with two example problems:

1. Calculating the nth fibonacci number
2. Checking if a string is a subsequence of another

## The Fibonacci Numbers

The **fibonacci numbers** form a sequence called the **fibonacci sequence**, where each number of the sequence is given by the sum of the two previous numbers in the sequence. The 0th fibonacci number is 0 and the 1st fibonacci number is 1. Here's the beginning of the fibonacci sequence:

    0, 1, 1, 2, 3, 5, 8, 13...

Again, each number is the sum of the two numbers right before it. For example, 13 is given by 5 + 8.

Our task will be to write a function that, given an input `n`, returns the nth fibonacci number.

In [32]:
def fib(n):
    """ Find the nth fibonacci number """
    return n if n < 2 else fib(n-2) + fib(n-1)

In [33]:
print(fib(5)) # 5
print(fib(6)) # 8
print(fib(7)) # 13 = 5 + 8

5
8
13


This is quite slow because:
1. It doesn't "remember" any of its answers.
2. It has to make many (recursive) function calls. 

Let's focus on number 1. Here's tree visualizing the recursion happening when we call `fib(4)`:

![alt text](fib.png "Fibonacci Recursion Tree")

Notice that it calls `fib(2)` twice! Hmm, what if we could "save" the result of `fib(2)` so we can avoid having to calculate it over and over? What data structure could we use to do this? We could use a `dictionary` or a `list`. 

Let's try it out with a dictionary called `memo`:

In [34]:
def fib_recursive_memo(n, memo = {0:0, 1:1}): # Abusing python's default mutable argument behavior!
    """ Find the nth fibonacci number """
    memo[n] = memo[n] if n in memo else fib_recursive_memo(n-2) + fib_recursive_memo(n-1)
    return memo[n]

In [35]:
print(fib_recursive_memo(5)) # 5
print(fib_recursive_memo(6)) # 8
print(fib_recursive_memo(7)) # 13 = 5 + 8

5
8
13


In [36]:
def fib_iter_memo(n, memo = [0,1]): # Abusing python's default mutable argument behavior!
    """ Find the nth fibonacci number """
    while n >= len(memo):
        memo.append(memo[-2] + memo[-1])
    return memo[n]

In [37]:
print(fib_iter_memo(5)) # 5
print(fib_iter_memo(6)) # 8
print(fib_iter_memo(7)) # 13 = 5 + 8

5
8
13


In [38]:
@lru_cache()
def fib_lru(n):
    return n if n in {0,1} else fib_lru(n-2) + fib_lru(n-1)

In [39]:
print(fib_lru(5)) # 5
print(fib_lru(6)) # 8
print(fib_lru(7)) # 13 = 5 + 8

5
8
13


In [40]:
# Test for correctness:
for i in range(20):
    results = [fib(i), fib_recursive_memo(i), fib_iter_memo(i), fib_lru(i)]
    assert all(results[0] == result for result in results)
    print(results[0])

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181


In [41]:
def time_fib(fib_func, inputs):
    start = time.time()
    for inp in inputs:
        fib_func(inp)
    return time.time() - start

In [42]:
FIB_FUNCS = [fib, fib_recursive_memo, fib_iter_memo, fib_lru]

inputs = [32, 31, 30]
time_taken = {}
for fib_func in FIB_FUNCS:
    time_taken[fib_func] = time_fib(fib_func, inputs)
    print(f"Called {fib_func.__name__} on inputs {inputs}.\n Time taken: {round(time_taken[fib_func], 6)}\n")

Called fib on inputs [32, 31, 30].
 Time taken: 1.575601

Called fib_recursive_memo on inputs [32, 31, 30].
 Time taken: 2.2e-05

Called fib_iter_memo on inputs [32, 31, 30].
 Time taken: 6e-06

Called fib_lru on inputs [32, 31, 30].
 Time taken: 9e-06



## Is Subsequence

A string `s` is a **subsequence** of a string `t` if `s` can be created by deleting some (or no) characters from `t`, without changing the order of the remaining characters. Note that this is different from a substring! For example:

- `"light"` is a subsequence, and a substring, of `"delightful"`
- `"elf"` is a subsequence, but NOT a substring, of `"delightful"`

**Task**: Write a function that checks if a string `s` is a subsequence of a string `t`.

In [43]:
def is_subsequence(s, t):
    """ Given a string s and a string t, return True if s is a subsequence of t, and False otherwise """
    if not s:
        return True
    i = 0
    for char in t:
        if i < len(s) and char == s[i]:
            i +=1
        if i == len(s):
            return True
    return False

In [44]:
assert is_subsequence("pset", "upset")
assert is_subsequence("woe", "wonderful")
assert is_subsequence("hello","Sphenopalatine ganglioneuralgia") # this means brain freeze
assert is_subsequence("","")
assert not is_subsequence("nah", "")

This works great when we are only calling `is_subsequence` a few times. But what if we needed to call `is_subsequence` many times, and perhaps sometimes with the same inputs? This would be especially expensive if `t` was long. It would be useful to save our answers so that we don't need to check the same pair of `s` and `t` twice. We can solve this by memoizing!

In [45]:
def is_subsequence_memo(s, t, memo={}):
    """ Given a string s and a string t, return True if s is a subsequence of t, and False otherwise """        
    if (s,t) in memo:
        return memo[(s, t)]
    if not s:
        return True
    i = 0
    for char in t:
        if i < len(s) and char == s[i]:
            i +=1
        if i == len(s):
            memo[(s,t)] = True
            return memo[(s,t)]
    memo[(s,t)] = False
    return memo[(s,t)]

In [46]:
assert is_subsequence_memo("pset", "upset")
assert is_subsequence_memo("woe", "wonderful")
assert is_subsequence_memo("hello","Sphenopalatine ganglioneuralgia") # this means brain freeze
assert is_subsequence_memo("","")
assert not is_subsequence_memo("nah", "")

In [51]:
# Demonstration of performance gain from memoization

t = "Sphenopalatine ganglioneuralgia"
s_strings = ["pset", "woe", "hello"]*10000

# Time is_sequence
start = time.time()
for s in s_strings:
    is_subsequence(s, t)
print(f"Time taken without memoization: {round(time.time() - start, 2)} seconds")

# Time is_sequence_memo
start = time.time()
for s in s_strings:
    is_subsequence_memo(s, t)
print(f"Time taken with memoization: {round(time.time() - start, 2)} seconds")

Time taken without memoization: 0.17 seconds
Time taken with memoization: 0.01 seconds
